# Projet de Python pour la data science 2024 : Prédiction de la demande de recharge de véhicules électriques à Paris
#### Encadrant : M. Victor Quétu 
#### Étudiants : Avner EL BAZ, Manuel Junior NDEFFO NKAMGUE, Daniel TRUSKINOVSKY

![borne-elect](/home/onyxia/work/images/borne-elect.jpg)

Dans une perspective éco-responsable d'adaptation des déplacements quotidiens, l'usage de la voiture électrique a le vent en poupe en France. En 2023, 17% des voitures achetées neuves étaient tout électriques et 9% étaient hybrides. En Île-de-France, les ventes de véhicules tout électriques ont augmenté de 60% entre 2022 et 2023, et celles de véhicules hybrides de 41%. 

Ainsi, la ville de Paris s'est dotée d'un réseau de bornes de recharge de véhicules, initialement créées pour les véhicules en libre-service Autolib' de 2011 jusqu'à l'arrêt du service en 2018. En 2021, le service Belib' est créé pour opérer le réseau de bornes et augmenter l'offre dans la capitale.

Nous étudions dans ce projet un jeu de données tiré de plusieurs API publiques de la ville de Paris :
- Belib' - Prises de recharge pour véhicules électriques - Données statiques, Paris OpenData.
- Belib' - Prises de recharge pour véhicules électriques - Disponibilité temps réel, Paris OpenData.
- Comptage routier - Données trafic issues des capteurs permanents, Paris OpenData.

Nous avons obtenu ce jeu de données sur la plateforme Challenge Data, gérée par le data lab de l'ENS Paris.

Ce jeu de données contient les informations de disponibilité des stations de recharge électrique à Paris du 25/11/2019 au 8/11/2020, donc avant la mise en place officielle du service Belib' (les stations appartiennent maintenant et réseau Belib' et leurs données sont bien enregistrées dans la même API), actualisées toutes les 15 min + autres données trafic routier à préciser.

Avec l'augmentation du nombre de véhicules électriques, il est intéressant de pouvoir prédire la demande de bornes de recharge selon la disponibilité en temps réel dans une certaine zone et l'état du trafic. Depuis 2020, de nombreuses nouvelles stations ont été implantées à Paris : Les API de la ville de Paris étant toujours actives, nous pourrons comparer nos prédictions à l'implantation réelle des stations de 2021 à 2024.


#### Importation des librairies

In [2]:
import numpy as np
import pandas as pd
import folium

#### Description des données

1) Belib' - Prises de recharge pour véhicules électriques - Disponibilité temps réel

Ce jeu de données recense le statut de chaque borne de recharge à Paris : Disponible, En Charge, Passive (branchée mais ne délivrant pas de courant), Hors ligne (ne transmettant pas d'information sur son statut, parmi les 3 précédents), Hors d'usage (en travaux). Les données sont actualisées toutes les 15 minutes sur la période concernée.

2) Belib' - Prises de recharge pour véhicules électriques - Données statiques

Ce jeu de données contient des informations sur les bornes de recharge : localisation géographique, adresse, identifiant sous la forme Sk-Tj pour la borne j de la station k, dispositifs de recharge disponibles.

3) Comptage routier : à voir





#### Importation des données

In [5]:
df_real_time=pd.read_csv("ytrain_NpxebDC.csv")
df_static=pd.read_csv("info_static.csv")

/tmp/ipykernel_89854/656849402.py:1: DtypeWarning: Columns (1,3,5,11,20,49,52,54,55,56,64,70,71,94,95,101,102,114,129,131,132,133,138,139,152,157,163,174,180,184,193,196,201,207,208,219,220,228,241,245,250,256,258,264,265,269,270,271,272,273) have mixed types. Specify dtype option on import or set low_memory=False.
  df_real_time=pd.read_csv("ytrain_NpxebDC.csv")


In [6]:
df_real_time.head(20)
df_static.head(20)

,t_id,s_id,latitude,longitude,street,city,zipcode,model
0,S84-T1,S84,48.863766,2.411770,104 RUE LOUIS LUMIERE,Paris,75020,"E/F, Combo, T2, CHAdeMO, T3"
1,S84-T3,S84,48.863766,2.411770,104 RUE LOUIS LUMIERE,Paris,75020,"CHAdeMO, T2, T3, E/F, Combo"
2,S86-T1,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,"CHAdeMO, Combo, T2, T3, E/F"
3,S86-T2,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,E/F
4,S86-T3,S86,48.890610,2.354230,27 RUE ORDENER,Paris,75018,"T3, Combo, CHAdeMO, E/F, T2"
5,S68-T1,S68,48.875465,2.360103,150 RUE DU FAUBOURG SAINT MARTIN,Paris,75010,"Combo, CHAdeMO, T3, T2, E/F"
6,S68-T3,S68,48.875465,2.360103,150 RUE DU FAUBOURG SAINT MARTIN,Paris,75010,"Combo, E/F, T3, CHAdeMO, T2"
7,S28-T1,S28,48.864719,2.359043,5 RUE DE REAUMUR,Paris,75003,"CHAdeMO, T2, Combo, E/F, T3"
8,S28-T3,S28,48.864719,2.359043,5 RUE DE REAUMUR,Paris,75003,"E/F, T3, CHAdeMO, Combo, T2"
9,S17-T3,S17,48.845265,2.411238,4 Rue Changarnier,Paris,75012,"CHAdeMO, T3, Combo, E/F, T2"


#### Traitement des données